In [1]:
import torch

import gym
from gym import spaces
from gym.utils.env_checker import check_env as gym_check_env

import stable_baselines3 as sb3
from stable_baselines3.common.env_checker import check_env as sb3_check_env

import numpy as np
from numpy import linalg
from sympy import sin, cos, pi  # 

from scipy.stats import vonmises_line

import inspect


def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]

def log(*argv):
    for arg in argv:
        print(f"-"*75)
        print(f"{retrieve_name(arg)}")
        print(f"content: ")
        print(arg)
        print(f"type: {type(arg)}")
        if isinstance(arg, np.ndarray) or isinstance(arg, torch.Tensor): 
            print(f"shape: {arg.shape}")
        elif isinstance(arg, list) or isinstance(arg, str) or isinstance(arg, dict):
            print(f"len: {len(arg)}")

In [2]:
x = np.array([[1, 2, 3]])
print(x.shape)
print(x.T.shape)
print(np.squeeze(x).shape)
print(np.squeeze(x, axis=0).shape)

x = np.array([[1234]])
print(np.squeeze(x))

(1, 3)
(3, 1)
(3,)
(3,)
1234


In [3]:
H_1 = np.random.normal(0, np.sqrt(0.5), (16, 4)) + 1j*np.random.normal(0, np.sqrt(0.5), (16, 4)) 
H_2 = np.random.normal(0, np.sqrt(0.5), (4, 16)) + 1j*np.random.normal(0, np.sqrt(0.5), (4, 16))  
H_3 = np.random.normal(0, np.sqrt(0.5), (4, 4)) + 1j*np.random.normal(0, np.sqrt(0.5), (4, 4))  
Phi = np.eye(16, dtype=complex)

print("H_1.shape, H_2.shape, H_3.shape, Phi.shape: ", H_1.shape, H_2.shape, H_3.shape, Phi.shape)


H_1_real, H_1_imag = np.real(H_1).reshape(1, -1), np.imag(H_1).reshape(1, -1)  # shape: (1, Ns * Nt)
H_2_real, H_2_imag = np.real(H_2).reshape(1, -1), np.imag(H_2).reshape(1, -1)  # shape: (1, Nk * Ns)
H_3_real, H_3_imag = np.real(H_3).reshape(1, -1), np.imag(H_3).reshape(1, -1)  # shape: (1, Nk * Nt)
Phi_real, Phi_imag = np.real(np.diag(Phi)).reshape(1, -1), np.imag(np.diag(Phi)).reshape(1, -1)  # shape: (1, Ns)

print("reshape(1, -1): ", H_1_real.shape, H_1_imag.shape, H_2_real.shape, H_2_imag.shape, \
      H_3_real.shape, H_3_imag.shape, Phi_real.shape, Phi_imag.shape)


H_1_real, H_1_imag = np.real(H_1).reshape(-1), np.imag(H_1).reshape(-1)  # shape: (1, Ns * Nt)
H_2_real, H_2_imag = np.real(H_2).reshape(-1), np.imag(H_2).reshape(-1)  # shape: (1, Nk * Ns)
H_3_real, H_3_imag = np.real(H_3).reshape(-1), np.imag(H_3).reshape(-1)  # shape: (1, Nk * Nt)
Phi_real, Phi_imag = np.real(np.diag(Phi)).reshape(-1), np.imag(np.diag(Phi)).reshape(-1) 

print("reshape(-1):    ", H_1_real.shape, H_1_imag.shape, H_2_real.shape, H_2_imag.shape, \
      H_3_real.shape, H_3_imag.shape, Phi_real.shape, Phi_imag.shape)

H_1.shape, H_2.shape, H_3.shape, Phi.shape:  (16, 4) (4, 16) (4, 4) (16, 16)
reshape(1, -1):  (1, 64) (1, 64) (1, 64) (1, 64) (1, 16) (1, 16) (1, 16) (1, 16)
reshape(-1):     (64,) (64,) (64,) (64,) (16,) (16,) (16,) (16,)


In [4]:
init_action = np.random.randint(low=0, high=2**8, size=16)
print(init_action.shape)

(16,)


In [5]:
observation = np.concatenate((H_1_real, H_1_imag, H_2_real, H_2_imag, H_3_real, H_3_imag, Phi_real, Phi_imag, init_action), axis=0)
# original shape when using np.hstack() is: (1, 336)
print(observation.shape)

(336,)
